In [65]:
using LinearAlgebra
using TensorKit
using KrylovKit
using JSON
using HDF5, JLD
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("kagome_load_tensor.jl")
include("kagome_CTMRG.jl")
include("kagome_model.jl")
include("kagome_IPESS.jl")
include("mps_algorithms\\ITEBD_algorithms.jl")
include("mps_algorithms\\TransfOp_decomposition.jl")
include("mps_algorithms\\PUMPS_algorithms.jl")
include("mps_algorithms\\ES_preliminary.jl")


try_ITEBD (generic function with 1 method)

In [59]:

#mps_virtual=SU₂Space(0=>1,1/2=>1,1=>1,3/2=>1);mps_phy=SU₂Space(1/2=>1);
mps_virtual=SU₂Space(0=>1,1/2=>1);mps_phy=SU₂Space(1/2=>1);
A=TensorMap(randn, mps_virtual*mps_virtual', mps_phy);
A=permute(A,(1,2,3,),())
display(space(A))


(Rep[SU₂](0=>1, 1/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1)' ⊗ Rep[SU₂](1/2=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [4]:
function HV_L_tensor(vl,A,mpo)
    if mpo==[]
        if numind(vl)==3
            @tensor vl[:]:=vl[-1,3,1]*A'[3,-2,2]*A[1,-3,2];
        elseif numind(vl)==2
            @tensor vl[:]:=vl[3,1]*A'[3,-2,2]*A[1,-3,2];
        end
    else
        if numind(vl)==5
            @tensor vl[:]:=vl[-1,7,5,3,1]*A'[7,-2,6]*mpo'[5,6,-3,4]*mpo[3,2,-4,4]*A[1,-5,2];
        elseif numind(vl)==4
            @tensor vl[:]:=vl[7,5,3,1]*A'[7,-1,6]*mpo'[5,6,-2,4]*mpo[3,2,-3,4]*A[1,-4,2];
        end
    end
    return vl
end



HV_L_tensor (generic function with 1 method)

In [62]:
HVfun1(x)=HV_L_tensor(x,A,[]);
vl_init = permute(TensorMap(randn, space(A,1), space(A,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
HVfun1(vl_init)
eu,ev=eigsolve(HVfun1, vl_init, 2,:LM,Arnoldi());
eu_S0=eu;
@assert maximum(abs.(eu)) == abs(eu[1])
eu=eu[1];
ev=ev[1];
display(eu_S0)

eu_allspin=eu_S0;
allspin=eu_S0*0;
vl_init = permute(TensorMap(randn, SU₂Space(1/2=>1)⊗space(A,1), space(A,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
if norm(vl_init)>0
    eu_S0d5,_=eigsolve(HVfun1, vl_init, 2,:LM,Arnoldi());
    eu_allspin=vcat(eu_allspin,eu_S0d5)
    allspin=vcat(allspin,0*eu_S0d5.+0.5)
end
display(eu_S0d5)

vl_init = permute(TensorMap(randn, SU₂Space(1=>1)⊗space(A,1), space(A,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
if norm(vl_init)>0
    eu_S1,_=eigsolve(HVfun1, vl_init, 2,:LM,Arnoldi());
    eu_allspin=vcat(eu_allspin,eu_S1)
    allspin=vcat(allspin,0*eu_S1.+1)
end
display(eu_S1)

vl_init = permute(TensorMap(randn, SU₂Space(3/2=>1)⊗space(A,1), space(A,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
if norm(vl_init)>0
    eu_S1d5,_=eigsolve(HVfun1, vl_init, 2,:LM,Arnoldi());
    eu_allspin=vcat(eu_allspin,eu_S1d5)
    allspin=vcat(allspin,0*eu_S1d5.+1.5)
end
display(eu_S1d5)

vl_init = permute(TensorMap(randn, SU₂Space(2=>1)⊗space(A,1), space(A,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
if norm(vl_init)>0
    eu_S2,_=eigsolve(HVfun1, vl_init, 2,:LM,Arnoldi());
    eu_allspin=vcat(eu_allspin,eu_S2)
    allspin=vcat(allspin,0*eu_S2.+2)
end
display(eu_S2)

eu_allspin_abs=abs.(eu_allspin);
@assert maximum(eu_allspin_abs)==eu_allspin_abs[1]

eu_allspin_abs_sorted=sort(eu_allspin_abs,rev=true);
eu_allspin_abs_sorted=eu_allspin_abs_sorted/eu_allspin_abs_sorted[1];
allspin=allspin[sortperm(eu_allspin_abs,rev=true)]
# display(eu_allspin_abs_sorted[2]/eu_allspin_abs_sorted[1])
# display(eu_allspin_abs_sorted[1:4])
# display(allspin[1:4])
#@assert eu_allspin_abs_sorted[2]/eu_allspin_abs_sorted[1]<0.99
if eu_allspin_abs_sorted[2]/eu_allspin_abs_sorted[1]>0.99
    display("Warn: dominant transfer matrix eigenvalue close to degenerate: "*string(eu_allspin_abs_sorted[2]/eu_allspin_abs_sorted[1]))
end


2-element Vector{ComplexF64}:
 -0.9658071430077358 + 0.0im
  0.9658071430077357 + 0.0im

2-element Vector{ComplexF64}:
 3.469446951953614e-18 + 0.6829287801391757im
 3.469446951953614e-18 - 0.6829287801391757im

1-element Vector{ComplexF64}:
 0.0 + 0.0im

3-element Vector{ComplexF64}:
  -3.2244798243011363 + 0.0im
   -1.618049340888668 + 0.0im
 -0.15734120086973033 + 0.0im

2-element Vector{ComplexF64}:
    1.8530140475885686 + 0.0im
 2.220446049250313e-15 + 0.0im

"Warn: dominant transfer matrix eigenvalue close to degenerate: 0.9999999999999999"

┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 2`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176


In [68]:
matwrite("LSM_theorem.mat", Dict(
    "A" => convert(Array,A)
); compress = false)


In [66]:
E=left_eigenvalue(A,A,2);
display(E)
@tensor AA[:]:=A[-1,1,-3]*A[1,-2,-4];
UAA=unitary(fuse(mps_phy⊗mps_phy), mps_phy'⊗mps_phy');
@tensor AA[:]:=AA[-1,-2,1,2]*UAA[-3,1,2]
EE=left_eigenvalue(AA,AA,2);
display(EE)


2-element Vector{ComplexF64}:
 -0.9658071430077358 + 0.0im
  0.9658071430077357 + 0.0im

1-element Vector{ComplexF64}:
 0.932783437484767 + 0.0im

In [69]:
HVfun1(x)=HV_L_tensor(x,AA,[]);
vl_init = permute(TensorMap(randn, space(AA,1), space(AA,1)), (1,2,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
HVfun1(vl_init)
eu,ev=eigsolve(HVfun1, vl_init, 20,:LM,Arnoldi());
eu_S0=eu;
@assert maximum(abs.(eu)) == abs(eu[1])
eu=eu[1];
ev=ev[1];
display(eu_S0)

eu_allspin=eu_S0;
allspin=eu_S0*0;
vl_init = permute(TensorMap(randn, SU₂Space(1/2=>1)⊗space(AA,1), space(AA,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
if norm(vl_init)>0
    eu_S0d5,_=eigsolve(HVfun1, vl_init, 20,:LM,Arnoldi());
    eu_allspin=vcat(eu_allspin,eu_S0d5)
    allspin=vcat(allspin,0*eu_S0d5.+0.5)
end
display(eu_S0d5)

vl_init = permute(TensorMap(randn, SU₂Space(1=>1)⊗space(AA,1), space(AA,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
if norm(vl_init)>0
    eu_S1,_=eigsolve(HVfun1, vl_init, 20,:LM,Arnoldi());
    eu_allspin=vcat(eu_allspin,eu_S1)
    allspin=vcat(allspin,0*eu_S1.+1)
end
display(eu_S1)

vl_init = permute(TensorMap(randn, SU₂Space(3/2=>1)⊗space(AA,1), space(AA,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
if norm(vl_init)>0
    eu_S1d5,_=eigsolve(HVfun1, vl_init, 20,:LM,Arnoldi());
    eu_allspin=vcat(eu_allspin,eu_S1d5)
    allspin=vcat(allspin,0*eu_S1d5.+1.5)
end
display(eu_S1d5)

vl_init = permute(TensorMap(randn, SU₂Space(2=>1)⊗space(AA,1), space(AA,1)), (1,2,3,),());# assume that the dominant eigenvector has total spin zero. If not, it will have three indeces and it's not Hermiitan.
if norm(vl_init)>0
    eu_S2,_=eigsolve(HVfun1, vl_init, 20,:LM,Arnoldi());
    eu_allspin=vcat(eu_allspin,eu_S2)
    allspin=vcat(allspin,0*eu_S2.+2)
end
display(eu_S2)


1-element Vector{ComplexF64}:
 0.9327834374847673 + 0.0im

1-element Vector{ComplexF64}:
 -0.4663917187423829 + 0.0im

1-element Vector{ComplexF64}:
 2.674144769472136e-16 + 0.0im

3-element Vector{ComplexF64}:
  -3.2244798243011363 + 0.0im
   -1.618049340888668 + 0.0im
 -0.15734120086973033 + 0.0im

2-element Vector{ComplexF64}:
    1.8530140475885686 + 0.0im
 2.220446049250313e-15 + 0.0im

┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 20`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 20`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176
┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 20`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176


In [70]:
"The Lanczos algorithm can't get degenerate eigenvalue, because in the degenerate subspace the matrix is identity. Imagine given an identity matrix, you act on a vector and the vector does't change." 

"The Lanczos algorithm can't get degenerate eigenvalue, because in the degenerate subspace the matrix is identity. Imagine given an identity matrix, you act on a vector and the vector does't change."

In [72]:
convert(Array, A)

3×3×2 Array{Float64, 3}:
[:, :, 1] =
  0.0       0.0  1.02157
 -0.668511  0.0  0.0
  0.0       0.0  0.0

[:, :, 2] =
  0.0       -1.02157  0.0
  0.0        0.0      0.0
 -0.668511   0.0      0.0

In [ ]:
decomposition: A[1,2:3,:], A[2:3,1,:], leads to two-sites unitcell


In [75]:
#show that the algorithm can't get degeneracy
function mx(x)
    return x;
end
idfun(x)=mx(x);
v0 = randn(3)

eu,ev=eigsolve(idfun, v0, 3,:LM,Arnoldi());
display(eu)
#to get degeneracy, the package should support providing more than one initial vector...


1-element Vector{ComplexF64}:
 1.0 + 0.0im

┌ Warning: Invariant subspace of dimension 1 (up to requested tolerance `tol = 1.0e-12`), which is smaller than the number of requested eigenvalues (i.e. `howmany == 3`); setting `howmany = 1`.
└ @ KrylovKit C:\Users\Lenovo\.julia\packages\KrylovKit\YPiz7\src\eigsolve\arnoldi.jl:176
